In [1]:
import tensorflow as tf
import os
import wave
import random
import pyaudio
import numpy as np
import pylab as plt

In [2]:
start_letter = 0
end_letter = 11
start_letter_asccii = start_letter
end_letter_asccii = end_letter + 1
letter_count = end_letter_asccii - start_letter_asccii

In [3]:
nchannels, sampwidth, framerate, nframes = None,None,None,None
def decodeWav(wav):
    global nchannels, sampwidth, framerate, nframes
    wf = wave.open(wav, "rb")
    nchannels, sampwidth, framerate, nframes = wf.getparams()[:4]
    data = wf.readframes(nframes)
    wf.close()
    soundBytes = np.fromstring(data, dtype=np.int16)
    soundBytes.shape = (-1, nchannels)
    graph = soundBytes[:, 0]
    return graph

In [4]:
def preprocessing(wavPath):
    soundBytes = decodeWav(wavPath)  
    soundBytes = normalization(soundBytes)
    return soundBytes  

In [5]:
def normalization(soundBytes):
    mean = soundBytes.mean()
    std = soundBytes.std()
    soundBytes = (soundBytes - mean)/std
    return soundBytes 

In [6]:
wavsPath = [] 
indexOfWavs = []
decodeWavs = []
labelOfWavs = [] 
seq_length = 1024

wavs_folder = r'D:\语音识别\念佛计数\端点检测训练集'
file_count = 0

for chrASCII in range(start_letter_asccii, end_letter_asccii):
        for root, dirs, files in os.walk(wavs_folder + "/" + str(chrASCII)):    
            print("letter {0} has files count: {1}".format( str(chrASCII), len(files)))
            file_count += len(files)
print("find {0} wave's files".format(file_count))

index = 0
for chrASCII in range(start_letter_asccii, end_letter_asccii):
    for root, dirs, files in os.walk(wavs_folder + "/" + str(chrASCII)):
        for file in files:
            if os.path.splitext(file)[1].lower() == '.wav':
                wavPath = os.path.join(root, file)
                print("\r" + "Feature extraction:{0}/{1}".format(index, file_count), end="", flush=True)
                label = chrASCII - start_letter_asccii           
                if label > 0:
                    label = 1
                decodeWavs.append(preprocessing(wavPath))
                wavsPath.append(wavPath)   
                labelOfWavs.append(label)
                indexOfWavs.append(index)
                index += 1
print("\r" + "Feature extraction:{0}/{1}".format(index, file_count), end="", flush=True)
print("\nFeature extraction finished")

letter 0 has files count: 11
letter 1 has files count: 3
letter 2 has files count: 3
letter 3 has files count: 3
letter 4 has files count: 3
letter 5 has files count: 3
letter 6 has files count: 2
letter 7 has files count: 2
letter 8 has files count: 2
letter 9 has files count: 2
letter 10 has files count: 2
letter 11 has files count: 2
find 38 wave's files
Feature extraction:2/38

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


Feature extraction:38/38
Feature extraction finished


In [7]:
decodeWavs_count = len(decodeWavs)
random.shuffle(indexOfWavs)
seq_data_x = []
seq_data_y = []
for si in range(decodeWavs_count):
    ix = indexOfWavs[si]
    seq_data_x.extend(decodeWavs[ix])
    seq_data_y.extend([labelOfWavs[ix]] * len(decodeWavs[ix]))

In [8]:
def getSamples(count):
    set_x = []
    set_y = []
    for i in range(count):
        frame = random.randint(0,len(seq_data_x) - 2 * seq_length)
        set_x.append(seq_data_x[frame:frame + seq_length])
        set_y.append(int(np.round(np.array(seq_data_y[frame + seq_length]).mean())))
    set_x = np.expand_dims(set_x, axis=-1)
    set_y = np.expand_dims(set_y, axis=-1)
    print("set_x shape:{} ,set_y shape:{}" .format(set_x.shape, set_y.shape))    
    return set_x, set_y

In [9]:
samples_count = 10000
data_x, data_y = getSamples(samples_count)
split_boundary = int(0.8 * samples_count)
train_x = data_x[:split_boundary]
train_y = data_y[:split_boundary]
test_x = data_x[split_boundary:]
test_y = data_y[split_boundary:]

train_count = np.shape(train_x)[0]
test_count = np.shape(test_x)[0]

tf_train_x = tf.data.Dataset.from_tensor_slices(train_x)
tf_train_y = tf.data.Dataset.from_tensor_slices(train_y)
tf_test_x = tf.data.Dataset.from_tensor_slices(test_x)
tf_test_y = tf.data.Dataset.from_tensor_slices(test_y)
tf_train_set = tf.data.Dataset.zip((tf_train_x, tf_train_y))
tf_test_set = tf.data.Dataset.zip((tf_test_x, tf_test_y))
(tf_train_set, tf_test_set)

set_x shape:(10000, 1024, 1) ,set_y shape:(10000, 1)


(<ZipDataset shapes: ((1024, 1), (1,)), types: (tf.float64, tf.int32)>,
 <ZipDataset shapes: ((1024, 1), (1,)), types: (tf.float64, tf.int32)>)

In [10]:
batch_size = 256
tf_train_set = tf_train_set.shuffle(samples_count).repeat().batch(batch_size)
tf_test_set = tf_train_set.batch(batch_size)

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(64, (3,), input_shape=(1024,1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D())
model.add(tf.keras.layers.Conv1D(128, (3,), activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1022, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 511, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 509, 128)          24704     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 91,521
Trainable params: 91,521
Non-trainable params: 0
____________________________________________________

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
steps_per_epoch = train_count//batch_size
validation_steps = test_count//batch_size

In [14]:
model.fit(tf_train_set, epochs=3, steps_per_epoch=steps_per_epoch, validation_data=tf_train_set, validation_steps=validation_steps)

Train for 31 steps, validate for 7 steps
Epoch 1/3
31/31 [==============================] - 17s 540ms/step - loss: 0.1512 - acc: 0.9555 - val_loss: 0.1321 - val_acc: 0.9833
Epoch 2/3
31/31 [==============================] - 19s 617ms/step - loss: 0.1135 - acc: 0.9700 - val_loss: 0.0975 - val_acc: 0.9632
Epoch 3/3
31/31 [==============================] - 18s 574ms/step - loss: 0.0932 - acc: 0.9763 - val_loss: 0.1300 - val_acc: 0.9743


In [15]:
MODEL_SAVE_PATH = r"D:\语音识别\念佛计数\h5\PFB_EndPointDetect.h5"
model.save(MODEL_SAVE_PATH)

In [ ]:
MODEL_SAVE_PATH = r"D:\语音识别\念佛计数\PFB_EndPointDetect.h5"
model = tf.keras.models.load_model(MODEL_SAVE_PATH)

In [16]:
import pyaudio
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 100
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [17]:
def epd_predict(data, seq_length):
    soundBytes = np.fromstring(np.array(data), dtype=np.int16)
    nsb = normalization(soundBytes).reshape(1,seq_length,1)   
    epd = int(np.round(model.predict(nsb)))    
    return epd

In [20]:
print("开始录音")
for t in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)    
    print(epd_predict(data, seq_length), end="")  
print("结束录音")

开始录音
0000

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


000000000000000000000111111111111111110000000001111111000111111111111101111110011111111111111111111111111101110000011111111111111011111001111111100000001111111111111111111111001111111111111

KeyboardInterrupt: 